# init data

In [2]:
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Any

seed = 18022004
np.random.seed(seed)

In [3]:
data_prefix: str = 'data'
repo_prefix: str = f'{data_prefix}/repos'

# data_name_template: str = '500_{}_sample_dataset.parquet'
# data_types = ['log', 'test', 'others']

# sampled_data_name: Dict[str, str] = {data_type: data_name_template.format(data_type) for data_type in data_types}

# log_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{sampled_data_name['log']}', engine = 'pyarrow')
# test_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{sampled_data_name['test']}', engine = 'pyarrow')
# others_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{sampled_data_name['others']}', engine = 'pyarrow')

data_name = 'migration_others_method_no_code.parquet'

# load data

In [4]:
data_df: pd.DataFrame = pd.read_parquet(f'{data_prefix}/{data_name}', engine = 'pyarrow')

# init parser

In [5]:
from tree_sitter import Language, Parser
import tree_sitter_java as tsjava
import difflib

from tqdm import tqdm

# Initialize the parser and set the Java language
JAVA_LANGUAGE = Language(tsjava.language())
parser = Parser(JAVA_LANGUAGE)

# preview

In [5]:
data_df.columns

Index(['id', 'fromLib', 'toLib', 'repoName', 'repoOwner', 'repoSplitName',
       'prevCommit', 'startCommit', 'endCommit', 'fileName',
       'startCommitChanges', 'endCommitChanges', 'total_methods_before',
       'total_methods_after', 'class_before', 'class_after',
       'method_before_name', 'method_after_name', 'method_before',
       'method_after', 'method_diff', 'method_before_modifiers',
       'method_after_modifiers', 'method_before_return_type',
       'method_after_return_type', 'method_before_parameters',
       'method_after_parameters', 'method_before_signature',
       'method_after_signature', 'method_before_signature_no_mod',
       'method_after_signature_no_mod'],
      dtype='object')

In [12]:
print(data_df.iloc[154550]['method_after'])

In [108]:
def print_tree(node, indent = 0):
    """
    Recursively prints the tree structure of a tree-sitter node.

    :param node: The current tree-sitter node.
    :param source_code: The source code being parsed, as bytes.
    :param indent: The current indentation level for pretty-printing.
    """
    # Extract node information
    node_type = node.type
    node_text = node.text.decode('utf-8')

    node_text = [line for line in node_text.split('\n') if line.strip()]

    # Print the node details with indentation
    print(f"{'  ' * indent}{node_type}: '{node_text}'")

    # Recursively print child nodes
    for child in node.children:
        print_tree(child, indent + 1)

def extract_method_details(node, class_name, source_code):
    method_details = {
        'class_name': class_name,
        'body': '',
        'modifiers': '',
        'return_type': '',
        'name': '',
        'parameters': '',
        'signature': '',
    }

    start_byte, end_byte = node.start_byte, node.end_byte
    method_details['body'] = source_code[start_byte:end_byte].decode('utf-8', errors = 'replace')

    for child in node.children:
        if (child.type == 'modifiers'):
            method_details['modifiers'] = ' '.join([modifier.text.decode('utf-8') for modifier in child.children])
        elif ('type' in child.type):  # Return type
            method_details['return_type'] = child.text.decode('utf-8')
        elif (child.type == 'identifier'):  # Method name
            method_details['name'] = child.text.decode('utf-8')
        elif (child.type.endswith('parameters')):  # Parameter list
            param_string = ', '.join([param.text.decode('utf-8') for param in child.children if param.type.endswith('parameter')])
            method_details['parameters'] = param_string

    method_details['signature'] = f'{method_details["modifiers"]} {method_details["return_type"]} {method_details["name"]}({method_details["parameters"]})'
    method_details['signature_no_mod'] = f'{method_details["return_type"]} {method_details["name"]}({method_details["parameters"]})'

    return method_details

def extract_methods_with_body(java_code):
    # print('java_code :',java_code)

    def has_errors(node):
        if node.type == 'ERROR':
            print(node.text.decode('utf-8'))
            return True
        return any(has_errors(child) for child in node.children)

    def dfs_find_methods(node: Any, encoded_code, class_context: List[str] = None, ) -> List[Dict[str, Any]]:
        '''Perform DFS to find all method_declaration nodes and their enclosing class hierarchy.'''
        if class_context is None:
            class_context = []

        method_details = []

        # If the node is a class, update the class context
        if node.type == 'class_declaration':
            class_name = None
            for child in node.children:
                if child.type == 'identifier':  # Class name
                    class_name = child.text.decode('utf-8')
                    break
            if class_name:
                class_context.append(class_name)

        # If the node is a method, extract its details with the full class hierarchy
        if node.type == 'method_declaration':
            full_class_name = '.'.join(class_context)  # Concatenate class names to show the hierarchy
            method_details.append(extract_method_details(node = node, class_name = full_class_name, source_code = encoded_code,))

        # Recursively process all children
        for child in node.children:
            method_details.extend(dfs_find_methods(child, encoded_code, class_context[:]))  # Pass a copy of class context

        # If the node is a class, pop the class name after processing its children
        if node.type == 'class_declaration' and class_context:
            class_context.pop()

        return method_details

    try:
        encoded_code = java_code.encode('utf-8')
        tree = parser.parse(encoded_code)
        root_node = tree.root_node

        if (has_errors(root_node)):
            raise Exception('Parsing errors found in the code')

        return dfs_find_methods(root_node, encoded_code)
    except Exception as e:
        print('Loi :' ,e)
        return None

# Function to remove comments
def remove_comments(java_code: str) -> str:
    # Parse the code
    tree = parser.parse(java_code.encode('utf-8'))
    root_node = tree.root_node

    # Gather ranges of comment nodes
    comment_ranges = []
    def visit_node(node):
        if node.type in {'line_comment', 'block_comment'}:
            comment_ranges.append((node.start_byte, node.end_byte, node.type))
        for child in node.children:
            visit_node(child)

    visit_node(root_node)

    # Remove comments by excluding their byte ranges
    result_code = bytearray(java_code, 'utf-8')
    for start, end, comment_type in reversed(comment_ranges):  # Reverse to avoid shifting indices
        if comment_type == 'block_comment':
            # Replace block comment with spaces
            result_code[start:end] = b' ' * (end - start + 1)
            # del result_code[start:end]
        else:
            # Remove line comments entirely
            result_code[start:end] = b' ' * (end - start + 1)
        # del result_code[start:end]

    return result_code.decode('utf-8').strip()

def diff_methods(methods_start, methods_end):
    '''
    Compare methods based on their full dictionaries (e.g., name, signature, body).
    '''
    # Normalize methods for comparison
    def normalize_methods(methods):
        res = []
        for method in methods:
            sub_method = {}

            sub_method['class_name'] = method['class_name'].strip()
            sub_method['name'] = method['name'].strip()
            sub_method['body'] = method['body'].strip()
            sub_method['modifiers'] = method['modifiers'].strip()
            sub_method['return_type'] = method['return_type'].strip()
            sub_method['parameters'] = method['parameters'].strip()
            sub_method['signature'] = method['signature'].strip()
            sub_method['signature_no_mod'] = method['signature_no_mod'].strip()

            res.append(sub_method)

        return res

    normalized_start = normalize_methods(methods_start)
    normalized_end = normalize_methods(methods_end)
    # normalized_start = [{key: method[key].strip() if isinstance(method[key], str) else method[key] for key in method} for method in methods_start]
    # normalized_end = [{key: method[key].strip() if isinstance(method[key], str) else method[key] for key in method} for method in methods_end]

    # Convert lists of methods to sets of frozensets for comparison
    set_start = set(frozenset(item.items()) for item in normalized_start)
    set_end = set(frozenset(item.items()) for item in normalized_end)

    # Determine differences
    removed_methods = [dict(items) for items in (set_start - set_end)]  # Methods in start but not in end
    added_methods = [dict(items) for items in (set_end - set_start)]    # Methods in end but not in start
    unchanged_methods = [dict(items) for items in (set_start & set_end)]  # Methods in both

    return {
        'removed': removed_methods,
        'added': added_methods,
        'unchanged': unchanged_methods,
    }

def check_same_methods(method1, method2):
    '''
    Check if two methods are the same based on their full dictionaries.
    '''
    return method1['signature_no_mod'] == method2['signature_no_mod']

In [117]:
for id in range(len(data_df)):
    # if (id != 8219):
    #     continue

    id = 8219
    sample = data_df.iloc[id]

    start_code = sample['startCode']
    end_code = sample['endCode']

    with open('before.txt', 'w') as f:
        f.write(start_code)
    with open('after.txt', 'w') as f:
        f.write(end_code)

    start_code_cleaned = remove_comments(start_code)
    end_code_cleaned = remove_comments(end_code)

    with open('before_no_comment.txt', 'w') as f:
        f.write(start_code_cleaned)
    with open('after_no_comment.txt', 'w') as f:
        f.write(end_code_cleaned)

    # with open('before.txt', 'w') as f:
    #     f.write(start_code)

    # print(start_code)

    method_start = extract_methods_with_body(start_code)
    method_end = extract_methods_with_body(end_code)

    # print(method_start)
    # print(method_end)

    # print(len(method_start))
    # print(len(method_end))

    break

# start_code

_(Class<? extends SubView> cls) {
Loi : Parsing errors found in the code
_(Class<? extends SubView> cls) {
Loi : Parsing errors found in the code
